In [4]:
from pathlib import Path
import os
import sys 
path_dcf = str(Path(os.path.abspath('.')).resolve().parent)
sys.path.append(path_dcf)
from config import *
from algorithms.oneshot_dcf import DCF
from utils import *
from algorithms.utils_dilated_tubules import *
import pandas as pd
import tifffile
import matplotlib.pyplot as plt
import cv2

/local/scratch/tmp/ipykernel_2146151/2424434615.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Choose a Slide from the dataset and a dilated tubule from the given slide

In [5]:
nb_slide = 1
nb_support_dilated_tubule = 8
nb_query_dilated_tubule = 20

### Instanciate the object DCF one-shot

In [6]:
dcf = DCF(n_epochs=1,
        nb_augment=1,
        learning_rate=0.,
        clip=1e-1,
        sigma=5,
        isolines = None,
        exponential_decay=0.999,
        thresh=1e-2,
        isoline_weights=None
        )

### Show the Support dilated tubule and the annotation of the support

In [7]:
pentagon_coords = torch.tensor([[
    [[1.0, 0.0],            # Vertex 1
     [0.309, 0.951],        # Vertex 2
     [-0.809, 0.588],       # Vertex 3
     [-0.809, -0.588],      # Vertex 4
     [0.309, -0.951]]       # Vertex 5
]])

scaled_pentagon_coords = (pentagon_coords * 0.5) + 0.5
scaled_pentagon_coords = scaled_pentagon_coords.view(1, 1, 5, 2)
contour_support = scaled_pentagon_coords.cuda()
numpy_array = contour_support.squeeze().cpu().detach().numpy()


In [8]:
img_support = (
    torch.randn(1 * 3 * 512 * 512).reshape((1, 3, 512, 512)).to(torch.float32)
).cuda()

### Fit the one shot DCF

In [9]:
dcf.fit(img_support,
        contour_support,
        augment = True)

100%|██████████| 1/1 [00:07<00:00,  7.99s/it]


### Predict on the query

In [10]:
# filename_img = row_to_filename(annotations.iloc[nb_query_dilated_tubule])
# img_test = tifffile.imread(os.path.join(path_images,filename_img))
# contour_init = contour_inits[filename_img]
# contour_init = preprocess_contour(contour_init,img_test)
contours, score, tots, energies = dcf.predict(img_support,
                                               contour_support)
# tots[tots == 0] = 1e100
# x = np.argmin(tots)
# contour_to_display = (np.expand_dims(contours[-1], 1)).astype(int)
# new_img = cv2.drawContours((((img_test/np.max(img_test))*255).astype(np.uint8)).copy(),
#                         [contour_to_display],
#                         0,
#                         color = 1,
#                         thickness=2)
# plt.imshow(new_img)
# plt.show()


[tensor([[[ 2.0978,  2.2976,  2.0623,  6.1110,  7.7447,  8.9694,  2.5970,
           6.4229,  7.9798,  1.7297, 11.5562,  7.3351,  1.7200,  4.5331,
           5.7592,  0.7183,  2.9372,  2.6203,  8.5892,  4.7331,  2.3013,
           1.0074,  4.2859,  4.4899,  6.9102,  4.3106,  1.3679,  1.6192,
           0.8201,  1.6414,  4.0609,  9.4274, 10.2939,  1.3866,  1.2334,
           9.7585, 11.7874,  4.8788,  2.3746,  7.5412,  6.8638,  9.5517,
           3.5640,  2.9724,  4.4034, 11.1614,  3.2071,  6.4183,  3.0445,
           3.6116,  3.3666,  2.3012,  3.3076,  9.5935,  2.4752,  3.8986,
           7.1500,  1.0064,  6.6986, 10.6571,  2.1836,  1.2648,  5.4788,
           7.5459]]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>), tensor([[[1.6673e+00, 1.9465e+01, 6.7021e+00, 5.2816e+00, 7.5697e+00,
          4.4494e+00, 8.7936e-02, 6.2484e+00, 2.0797e+01, 3.9062e+00,
          1.0146e+01, 8.5369e-01, 6.1374e+00, 3.0114e+00, 1.2796e+00,
          6.6763e-03, 2.0878e+00, 4.0427e+01, 4.3224e+00, 

### This cell save the images of the evolution of the contour over time in folder_images_paper

In [ ]:
import matplotlib
cmap = matplotlib.cm.get_cmap('cool')
 feedback a
colors = [cmap(u)[:-1] for u in np.linspace(0,1,dcf.n_epochs+1)]
colors = (np.array(colors)*255).astype(np.int32).tolist()

tots = []
imgs = []

for i, contour in enumerate(contours):

    contour_to_display = (np.expand_dims(contour, 1)).astype(int)
    
    new_img = cv2.drawContours((((img_test/np.max(img_test))*255).astype(np.uint8)).copy(),
                               [contour_to_display],
                               0,
                               color = colors[0],
                               thickness=3)
    plt.imsave(os.path.join('folder_images_paper',filename_img.split('.')[0]+'_'+str(i)+'.png'),new_img)


### This cell plot the evolution of the energies by (isoline, scale) over time

In [1]:
# ener = np.squeeze(energies.cpu().detach().numpy())                                                
# isolines = dcf.isolines.cpu().detach().numpy()
# for scale in range(5):
#     for index, isoline in enumerate(isolines):
#         plt.plot(np.arange(dcf.n_epochs),ener[:,scale,index],label = 'isoline '+str(isoline))
#     plt.title('evolution of the energies at scale: ' +str(scale+1))
#     plt.xlabel('step')
#     plt.ylabel('energie value')
#     plt.legend()
#     plt.show()


### This cell plot the repartition of the annotation over the different slides

In [ ]:
# annotations = pd.read_csv(os.path.join(path_annotations,'annotations.csv'), index_col=0)
# annotations = annotations.replace(['dilated_tubule','fake_tubule'], [1,0])
# dt_annotations = annotations[['image','term']].groupby('image').sum()
# dt_annotations.columns = ['number of dilated tubules']
# all_annotations = annotations[['image','term']].groupby('image').count()
# all_annotations.columns = ['number of annotations']

# dataset_stats = pd.concat([dt_annotations,all_annotations], axis = 1)
# dataset_stats['number of false dilated_tubule'] = dataset_stats['number of annotations'] - dataset_stats['number of dilated tubules']
# dataset_stats['image'] = dataset_stats.index
# dataset_stats = dataset_stats.reset_index(drop = True)
# dataset_stats['image'] = dataset_stats['image'].apply(lambda x: x.split('.')[0])
# dataset_stats
# # dataset_stats['image'] = dataset_stats.index.apply(lambda x : x.split('.')[0])

# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.set(font_scale=1)

# plt.figure(figsize = (10,10))
# # plotting columns

# ax = sns.barplot(x=dataset_stats['image'],
#                  y=dataset_stats['number of annotations'],
#                  color = np.array([207,105,138])/255,
#                  label = 'n° of annotations')

# ax = sns.barplot(x=dataset_stats['image'],
#                  y=dataset_stats['number of dilated tubules'],
#                  color = np.array([137,44,80])/255,
#                  label = 'n° of dilated tubules')
 
# # renaming the axes
# ax.set_xlabel("Slide",fontsize = 20)
# ax.set_ylabel("N° of instances",fontsize = 20)
# plt.legend(fontsize = 20)

# # visualizing illustration
# plt.show()
